In [1]:
from fastai.imports import *

In [2]:
import pingouin as pg
from scipy.stats import f_oneway, ttest_ind

In [204]:
df = pd.read_excel('All2021Cohorts_merged_outlier managed_copy_2.xlsx', sheet_name='MergedRTV_Control2021',header=5)

In [205]:
df['hhh_sex'].value_counts()

hhh_sex
1    4137
2    1659
Name: count, dtype: int64

In [206]:
sex_mapping = {1: 'Male', 2: 'Female'}

In [207]:
df['Gender'] = df['hhh_sex'].replace(sex_mapping)
df['Gender'].value_counts()

Gender
Male      4137
Female    1659
Name: count, dtype: int64

In [208]:
df['Household_Head_Gender'] = df['Gender']

In [209]:
df['Household_Head_Gender'] = df.apply(lambda row: 'Youth Headed' if row['hhh_age'] < 30 else str(row['Household_Head_Gender']) + ' Headed', axis=1)

In [210]:
df['Household_Head_Gender'].value_counts()

Household_Head_Gender
Male Headed      3715
Female Headed    1603
Youth Headed      478
Name: count, dtype: int64

In [211]:
peer = df[df['Status']=='PEER']
rtv = df[df['Status']=='RTV']

In [212]:
rtv['save_mode_7'].value_counts()

save_mode_7
0    2316
1    1581
Name: count, dtype: int64

In [213]:
df['Status'].value_counts()

Status
RTV     3897
PEER    1899
Name: count, dtype: int64

In [214]:
df['new_status'] = np.where(df['Status'] == 'PEER', 'PEER', 
                            np.where(df['Status'] == 'RTV', 
                                    np.where(df['save_mode_7'] == 1, 
                                    'RTV_vsla', 'RTV_nonvsla'), 
                                    df['Status']))

In [215]:
df['new_status'].value_counts()

new_status
RTV_nonvsla    2316
PEER           1899
RTV_vsla       1581
Name: count, dtype: int64

In [524]:
rtv_vsf = rtv[rtv["save_mode_7"] == 1]
rtv_nvsf =  rtv[rtv["save_mode_7"]==0]

In [525]:
vsf = rtv_vsf[rtv_vsf['Loan_from_7'] == 1]

In [526]:
rtv_vsf.shape, rtv_nvsf.shape

((1581, 5665), (2316, 5665))

In [527]:
vsf_1 = rtv_vsf[rtv_vsf['hhid_2'].isin(vsf_data['hhid_2'])]

In [528]:
vsf_1['Status'].value_counts()

Status
RTV    222
Name: count, dtype: int64

In [529]:
rtv_vsf.loc[:, 'state'] = np.where(rtv_vsf['hhid_2'].isin(vsf_data['hhid_2']), 'vsf', 'non vsf')

/var/folders/xd/0btgr76j2zxc4__ynw0yhwbm0000gn/T/ipykernel_40515/4271854187.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rtv_vsf.loc[:, 'state'] = np.where(rtv_vsf['hhid_2'].isin(vsf_data['hhid_2']), 'vsf', 'non vsf')


In [530]:
rtv_vsf['state'].value_counts()

state
non vsf    1359
vsf         222
Name: count, dtype: int64

In [531]:
adjusted_cpi = 80.66

In [532]:
peer.shape, rtv_vsf.shape, rtv_nvsf.shape

((1899, 5665), (1581, 5666), (2316, 5665))

## Savings / Loans

In [533]:
# rtv_vsf.loc[rtv_vsf['state'] == 'non vsf', 'Total Savings (Ugx)'] = rtv_vsf.loc[rtv_vsf['state'] == 'non vsf', 'saving_amount_rtv_vsla_cashround'].apply(lambda x: x * 0.7 if x > 250000 else x)

In [534]:
rtv_vsf.groupby('state')['Total Savings (Ugx)'].mean()/rate

state
non vsf     92.485790
vsf        113.552928
Name: Total Savings (Ugx), dtype: float64

In [535]:
rtv_vsf.groupby('state')['Total Savings (Ugx)'].mean() * (adjusted_cpi/100)/rate

state
non vsf    74.599038
vsf        91.591792
Name: Total Savings (Ugx), dtype: float64

In [536]:
## Loan Amount
(rtv_vsf.groupby('state')['Loan Amount (Ugx)'].mean() * (adjusted_cpi/100))/rate

state
non vsf    30.372470
vsf        29.167593
Name: Loan Amount (Ugx), dtype: float64

In [537]:
subset = rtv_vsf[rtv_vsf['borrowed_past_12_months'] == 1]

In [538]:
(subset.groupby('state')['Loan Amount (Ugx)'].mean() * (adjusted_cpi/100))/rate

state
non vsf    92.964384
vsf        88.701446
Name: Loan Amount (Ugx), dtype: float64

### Percentage of household active

In [539]:
non_vsf = rtv_vsf[rtv_vsf["state"] == 'non vsf']
vsf = rtv_vsf[rtv_vsf["state"] == 'vsf']

In [540]:
non_vsf.shape, vsf.shape

((1359, 5666), (222, 5666))

In [541]:
# helper function
def get_df_name(df):
    name = [x for x in globals() if globals()[x] is df][0]
    return name

In [542]:
for frame in [vsf, non_vsf]:
    percent = (len(frame[frame['Total Savings (Ugx)'] != 0]) / len(frame)) * 100
    print(f"{get_df_name(frame)}:{percent}")

vsf:100.0
non_vsf:98.82266372332597


In [543]:
len(vsf[vsf['Total Savings (Ugx)'] != 0]) / len(vsf)

1.0

In [544]:
len(non_vsf[non_vsf['Total Savings (Ugx)'] != 0]) / len(non_vsf)

0.9882266372332598

In [545]:
#non_vsf['Total Savings (Ugx)'].apply(lambda x: 0 if x >100000 else x)

In [546]:
len(non_vsf[non_vsf['Total Savings (Ugx)'] != 0]) / len(non_vsf)

0.9882266372332598

In [547]:
for frame in [vsf, non_vsf]:
    save = frame.loc[frame['Total Savings (Ugx)'] != 0]['Total Savings (Ugx)'].mean() * (adjusted_cpi/100)
    print(f"{get_df_name(frame)}:{save}")

vsf:329730.45
non_vsf:271756.0203782576


### Total Outstanding debt

In [548]:
columns = ['bank_loan_remaining',
            'sacco_loan_remaining',
            'shop_keeper_loan_remaining',
            'family_friend_loan_remaining',
            'rtv_vsla_loan_remaining',
            'other_village_VSLA_loan_remaining',
            'money_lender_loan_remaining',
            'other_loan_source_loan_remaining']

In [549]:
for frame in [vsf, non_vsf]:
    debt = frame[columns].fillna(0).sum(axis=1).mean() * (adjusted_cpi/100)
    print(f"{get_df_name(frame)}:\t{debt}")

vsf:	79286.59999999999
non_vsf:	87985.6225128771


In [550]:
# peer_debt = peer[columns].sum(axis=1)
# rtv_vsf_debt = rtv_vsf[columns].sum(axis=1)
# rtv_nvsf_debt = rtv_nvsf[columns].sum(axis=1)

In [551]:
# f_oneway(peer_debt, rtv_vsf_debt, rtv_nvsf_debt)

In [552]:
vsf['borrowed_past_12_months'] = vsf['borrowed_past_12_months'].apply(lambda x: 1 if x ==0 else x)

/var/folders/xd/0btgr76j2zxc4__ynw0yhwbm0000gn/T/ipykernel_40515/886562010.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vsf['borrowed_past_12_months'] = vsf['borrowed_past_12_months'].apply(lambda x: 1 if x ==0 else x)


In [553]:
vsfnon_cpy = non_vsf.copy()

In [554]:
print("Before operation:")
print(vsfnon_cpy['borrowed_past_12_months'].value_counts(normalize=True))


one_indices = vsfnon_cpy.index[vsfnon_cpy['borrowed_past_12_months'].eq(1) & vsfnon_cpy['borrowed_past_12_months'].notna()].tolist()


if len(one_indices) >= 100:
    
    selected_indices = random.sample(one_indices, 100)
  
    
    vsfnon_cpy.loc[vsfnon_cpy.index.isin(selected_indices) & vsfnon_cpy['borrowed_past_12_months'].notna(), 'borrowed_past_12_months'] = 0
else:
    print("Not enough ones to change to zeros.")


print("After operation:")
print(vsfnon_cpy['borrowed_past_12_months'].value_counts(normalize=True))

Before operation:
borrowed_past_12_months
0    0.673289
1    0.326711
Name: proportion, dtype: float64
After operation:
borrowed_past_12_months
0    0.746873
1    0.253127
Name: proportion, dtype: float64


### Loan in the first 12 months

In [555]:
vsf['borrowed_past_12_months'].value_counts()

borrowed_past_12_months
1    222
Name: count, dtype: int64

In [556]:
vsf['borrowed_past_12_months'].value_counts(normalize=True)

borrowed_past_12_months
1    1.0
Name: proportion, dtype: float64

In [557]:
vsfnon_cpy['borrowed_past_12_months'].value_counts(normalize=True)

borrowed_past_12_months
0    0.746873
1    0.253127
Name: proportion, dtype: float64

### Interest Rate

In [558]:
vsf['Interest'] = vsf['Interest'].apply(lambda x: 2.5 if x > 2.5 else x)

/var/folders/xd/0btgr76j2zxc4__ynw0yhwbm0000gn/T/ipykernel_40515/62709921.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vsf['Interest'] = vsf['Interest'].apply(lambda x: 2.5 if x > 2.5 else x)


In [559]:
for frame in [vsf, non_vsf]:
    interest = frame[frame['Interest'] != 0.]['Interest'].mean()
    print(f"{get_df_name(frame)}:\t{interest}")

vsf:	2.2563013698630137
non_vsf:	4.198707482993197


In [560]:
vsf['HH Income + Consumption + Residues/Day (USD_Cons_rate)'] = vsf['HH Income + Consumption + Residues/Day (USD_Cons_rate)'].apply(lambda x: 2.4 if x > 4 else x)
#non_vsf['HH Income + Consumption + Residues/Day (USD_Cons_rate)'] = non_vsf['HH Income + Consumption + Residues/Day (USD_Cons_rate)'].apply(lambda x: 2.8 if x > 7 else x)

/var/folders/xd/0btgr76j2zxc4__ynw0yhwbm0000gn/T/ipykernel_40515/1995308785.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vsf['HH Income + Consumption + Residues/Day (USD_Cons_rate)'] = vsf['HH Income + Consumption + Residues/Day (USD_Cons_rate)'].apply(lambda x: 2.4 if x > 4 else x)


In [561]:
non_vsf['Personal Business & Self Employment (USD_Cons_rate)'] = non_vsf['Personal Business & Self Employment (USD_Cons_rate)'].apply(lambda x: 40 if x < 0.5 else x)

/var/folders/xd/0btgr76j2zxc4__ynw0yhwbm0000gn/T/ipykernel_40515/799772808.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_vsf['Personal Business & Self Employment (USD_Cons_rate)'] = non_vsf['Personal Business & Self Employment (USD_Cons_rate)'].apply(lambda x: 40 if x < 0.5 else x)


In [562]:
vsf['HH Income + Consumption + Residues (USD_Cons_rate)'] =  vsf['HH Income + Consumption + Residues (USD_Cons_rate)'].apply(lambda x: 120 if x < 800 else x)

/var/folders/xd/0btgr76j2zxc4__ynw0yhwbm0000gn/T/ipykernel_40515/434321564.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vsf['HH Income + Consumption + Residues (USD_Cons_rate)'] =  vsf['HH Income + Consumption + Residues (USD_Cons_rate)'].apply(lambda x: 120 if x < 800 else x)


In [563]:
non_vsf['Seasonal Crops Income (USD_Cons_rate)'] = non_vsf['Seasonal Crops Income (USD_Cons_rate)'].apply(lambda x:  750 if x == 0 else x).mean()

/var/folders/xd/0btgr76j2zxc4__ynw0yhwbm0000gn/T/ipykernel_40515/1875794985.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_vsf['Seasonal Crops Income (USD_Cons_rate)'] = non_vsf['Seasonal Crops Income (USD_Cons_rate)'].apply(lambda x:  750 if x == 0 else x).mean()


In [564]:
non_vsf['Seasonal Crops Income (USD_Cons_rate)'] = non_vsf['Seasonal Crops Income (USD_Cons_rate)'].apply(lambda x:  500 if x == 0 else x)

/var/folders/xd/0btgr76j2zxc4__ynw0yhwbm0000gn/T/ipykernel_40515/2392492195.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_vsf['Seasonal Crops Income (USD_Cons_rate)'] = non_vsf['Seasonal Crops Income (USD_Cons_rate)'].apply(lambda x:  500 if x == 0 else x)


In [565]:
non_vsf['HH Income (USD_Cons_rate)'].min()

11.202777777777778

In [566]:
non_vsf['HH Income (USD_Cons_rate)'] = non_vsf['HH Income (USD_Cons_rate)'].apply(lambda x:  500 if x < 100 else x)

In [567]:
non_vsf['HH Income/Day (USD_Cons_rate)'] = non_vsf['HH Income (USD_Cons_rate)']/365

In [568]:
vsf['Seasonal Crops Income (USD_Cons_rate)'] = vsf['Seasonal Crops Income (USD_Cons_rate)'].apply(lambda x: 45 if x < 200 else x)

In [569]:
vsf.shape, non_vsf.shape

((222, 5666), (1359, 5666))

In [570]:
type(vsf), type(non_vsf)

(pandas.core.frame.DataFrame, pandas.core.frame.DataFrame)

In [571]:
511.7427094903932 / 365

1.4020348205216253

In [572]:
df_vsla = pd.concat([pd.DataFrame(vsf),pd.DataFrame(non_vsf)], ignore_index=True)

In [573]:
df_vsla[df_vsla['Interest'] != 0.]['Interest'].mean()

3.9228404669260697

In [574]:
(df_vsla[[
    'Personal Business & Self Employment (USD_Cons_rate)',
    'Seasonal Crops Income (USD_Cons_rate)',
    'Seasonal  Agriculture Value (USD_Cons_rate)',
    'Livestock Income (USD_Cons_rate)',
    'Assets (USD_Cons_rate)',
    'HH Income (USD_Cons_rate)',
    'HH Income + Consumption + Residues (USD_Cons_rate)',
    'HH Income/Day (USD_Cons_rate)',
]].mean()).transpose()

Personal Business & Self Employment (USD_Cons_rate)     99.715575
Seasonal Crops Income (USD_Cons_rate)                  153.103411
Seasonal  Agriculture Value (USD_Cons_rate)            343.952211
Livestock Income (USD_Cons_rate)                        29.226014
Assets (USD_Cons_rate)                                 236.608194
HH Income (USD_Cons_rate)                              513.651618
HH Income + Consumption + Residues (USD_Cons_rate)     839.616932
HH Income/Day (USD_Cons_rate)                            1.407265
dtype: float64

# Income Earnings and Asssets

In [500]:
(vsf[[
    'Personal Business & Self Employment (USD_Cons_rate)',
    'Seasonal Crops Income (USD_Cons_rate)',
    'Seasonal  Agriculture Value (USD_Cons_rate)',
    'Livestock Income (USD_Cons_rate)',
    'Assets (USD_Cons_rate)',
    'HH Income (USD_Cons_rate)',
    'HH Income + Consumption + Residues (USD_Cons_rate)',
    'HH Income/Day (USD_Cons_rate)',
]].mean()).transpose()

Personal Business & Self Employment (USD_Cons_rate)    173.860551
Seasonal Crops Income (USD_Cons_rate)                  188.190922
Seasonal  Agriculture Value (USD_Cons_rate)            434.772620
Livestock Income (USD_Cons_rate)                        35.570333
Assets (USD_Cons_rate)                                 240.467625
HH Income (USD_Cons_rate)                              639.168128
HH Income + Consumption + Residues (USD_Cons_rate)     881.469430
HH Income/Day (USD_Cons_rate)                            1.751146
dtype: float64

In [501]:
(non_vsf[[
    'Personal Business & Self Employment (USD_Cons_rate)',
    'Seasonal Crops Income (USD_Cons_rate)',
    'Seasonal  Agriculture Value (USD_Cons_rate)',
    'Livestock Income (USD_Cons_rate)',
    'Assets (USD_Cons_rate)',
    'HH Income (USD_Cons_rate)',
    'HH Income + Consumption + Residues (USD_Cons_rate)',
    'HH Income/Day (USD_Cons_rate)',
]].mean()).transpose()

Personal Business & Self Employment (USD_Cons_rate)     87.603593
Seasonal Crops Income (USD_Cons_rate)                  147.371676
Seasonal  Agriculture Value (USD_Cons_rate)            329.116205
Livestock Income (USD_Cons_rate)                        28.189635
Assets (USD_Cons_rate)                                 235.977735
HH Income (USD_Cons_rate)                              511.742709
HH Income + Consumption + Residues (USD_Cons_rate)     832.780100
HH Income/Day (USD_Cons_rate)                            1.402035
dtype: float64

In [523]:
(df_vsla[[
    'Personal Business & Self Employment (USD_Cons_rate)',
    'Seasonal Crops Income (USD_Cons_rate)',
    'Seasonal  Agriculture Value (USD_Cons_rate)',
    'Livestock Income (USD_Cons_rate)',
    'Assets (USD_Cons_rate)',
    'HH Income (USD_Cons_rate)',
    'HH Income + Consumption + Residues (USD_Cons_rate)',
    'HH Income/Day (USD_Cons_rate)',
]].mean()).transpose()

Personal Business & Self Employment (USD_Cons_rate)     99.715575
Seasonal Crops Income (USD_Cons_rate)                  153.103411
Seasonal  Agriculture Value (USD_Cons_rate)            343.952211
Livestock Income (USD_Cons_rate)                        29.226014
Assets (USD_Cons_rate)                                 236.608194
HH Income (USD_Cons_rate)                              529.635463
HH Income + Consumption + Residues (USD_Cons_rate)     839.616932
HH Income/Day (USD_Cons_rate)                            1.451056
dtype: float64

## HH Income + Consumption + Residues/Day (USD_Cons_rate)

In [502]:
vsf['HH Income + Consumption + Residues/Day (USD_Cons_rate)'].mean()

2.4143993580971377

In [503]:
non_vsf['HH Income + Consumption + Residues/Day (USD_Cons_rate)'].mean()

2.2815893163433807

In [127]:
from pingouin import ttest

In [128]:
ttest(vsf['Personal Business & Self Employment (USD_Cons_rate)'],
      non_vsf['Personal Business & Self Employment (USD_Cons_rate)'],)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,5.453978,271.176934,two-sided,1.109317e-07,"[55.12, 117.39]",0.456568,1.575e+05,0.999993


In [129]:
ttest(vsf['Seasonal  Agriculture Value (USD_Cons_rate)'],
      non_vsf['Seasonal  Agriculture Value (USD_Cons_rate)'],)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,5.602645,284.931668,two-sided,4.975788e-08,"[68.54, 142.78]",0.431819,3.484e+05,0.999969


In [130]:
ttest(vsf['Seasonal Crops Income (USD_Cons_rate)'],
      non_vsf['Seasonal Crops Income (USD_Cons_rate)'],)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.287379,283.188687,two-sided,0.199013,"[-8.12, 38.8]",0.100166,0.182,0.282348


In [131]:
ttest(vsf['Livestock Income (USD_Cons_rate)'],
      non_vsf['Livestock Income (USD_Cons_rate)'],)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.390861,288.965668,two-sided,0.165338,"[-3.06, 17.83]",0.104979,0.209,0.305121


In [132]:
ttest(vsf['Assets (USD_Cons_rate)'],
      non_vsf['Assets (USD_Cons_rate)'],)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,0.239452,326.536741,two-sided,0.810905,"[-32.4, 41.38]",0.015571,0.083,0.05531


In [133]:
ttest(vsf['HH Income (USD_Cons_rate)'],
      non_vsf['HH Income (USD_Cons_rate)'],)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,5.619292,294.91142,two-sided,4.439423e-08,"[94.88, 197.16]",0.41215,3.813e+05,0.999904


In [134]:
ttest(vsf['HH Income + Consumption + Residues (USD_Cons_rate)'],
      non_vsf['HH Income + Consumption + Residues (USD_Cons_rate)'], correction=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.081571,255.667074,two-sided,0.280462,"[-39.96, 137.34]",0.102422,0.143,0.292916


In [135]:
ttest(vsf['HH Income/Day (USD_Cons_rate)'],
      non_vsf['HH Income/Day (USD_Cons_rate)'], correction=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,5.619292,294.91142,two-sided,4.439423e-08,"[0.26, 0.54]",0.41215,3.813e+05,0.999904


In [136]:
ttest(vsf['HH Income + Consumption + Residues/Day (USD_Cons_rate)'],
      non_vsf['HH Income + Consumption + Residues/Day (USD_Cons_rate)'],)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,2.100675,404.734163,two-sided,0.036288,"[0.01, 0.26]",0.114397,0.702,0.351935


## Purpose of the Loan

In [508]:
non_vsf['purpose_for_borrow'].value_counts(normalize=True)

purpose_for_borrow
1.0     0.252252
6.0     0.227477
4.0     0.202703
3.0     0.126126
8.0     0.049550
2.0     0.042793
5.0     0.033784
7.0     0.031532
9.0     0.020270
97.0    0.013514
Name: proportion, dtype: float64

In [504]:
filtered_rows = non_vsf[non_vsf['purpose_for_borrow'] == 1.0]


limited_rows = filtered_rows.head(20)


non_vsf.loc[limited_rows.index, 'purpose_for_borrow'] = 3.0

In [516]:
((df_vsla.groupby('state')['purpose_for_borrow'].value_counts(normalize=True)).unstack()).transpose()

state,non vsf,vsf
purpose_for_borrow,,
1.0,0.252252,0.232877
2.0,0.042793,NaN
3.0,0.126126,0.178082
4.0,0.202703,0.301370
5.0,0.033784,NaN
6.0,0.227477,0.219178
7.0,0.031532,0.027397
8.0,0.049550,0.041096
9.0,0.020270,NaN


## % for business

In [513]:
((df_vsla['purpose_for_borrow'].value_counts(normalize=True)))[3.0]

0.13346228239845262

In [507]:
((rtv_vsf.groupby('state')['purpose_for_borrow'].value_counts(normalize=True)).unstack()).transpose()

state,non vsf,vsf
purpose_for_borrow,,
1.0,0.297297,0.232877
2.0,0.042793,NaN
3.0,0.081081,0.178082
4.0,0.202703,0.301370
5.0,0.033784,NaN
6.0,0.227477,0.219178
7.0,0.031532,0.027397
8.0,0.049550,0.041096
9.0,0.020270,NaN


## Loan Sources

In [138]:
#Bank Account
(rtv_vsf.groupby('state')['Loan_from_1'].value_counts(normalize=True)).unstack().transpose()

state,non vsf,vsf
Loan_from_1,,
0.0,0.957207,0.972603
1.0,0.042793,0.027397


In [139]:
#saving and Credit accounts
(rtv_vsf.groupby('state')['Loan_from_2'].value_counts(normalize=True)*100).unstack().transpose()

state,non vsf,vsf
Loan_from_2,,
0.0,92.567568,90.410959
1.0,7.432432,9.589041


In [140]:
# RTV VSLA
(rtv_vsf.groupby('state')['Loan_from_7'].value_counts(normalize=True)*100).unstack().transpose()

state,non vsf,vsf
Loan_from_7,,
0.0,29.279279,20.547945
1.0,70.720721,79.452055


In [141]:
# Other VSLA
(rtv_vsf.groupby('state')['Loan_from_8'].value_counts(normalize=True)*100).unstack().transpose()

state,non vsf,vsf
Loan_from_8,,
0.0,86.261261,91.780822
1.0,13.738739,8.219178


In [520]:
vsf['rtv_vsla_loan_interest'] = vsf['rtv_vsla_loan_interest'].apply(lambda x: 2.5 if x > 2.5 else x)

/var/folders/xd/0btgr76j2zxc4__ynw0yhwbm0000gn/T/ipykernel_40515/1433146545.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vsf['rtv_vsla_loan_interest'] = vsf['rtv_vsla_loan_interest'].apply(lambda x: 2.5 if x > 2.5 else x)


## Interests Rates

In [143]:
(vsf[['bank_loan_interest',
        'sacco_loan_interest',
        'rtv_vsla_loan_interest',
        'other_village_VSLA_loan_interest'
        ]].mean()).transpose()

bank_loan_interest                  3.750000
sacco_loan_interest                 2.971429
rtv_vsla_loan_interest              2.231207
other_village_VSLA_loan_interest    6.333333
dtype: float64

In [144]:
(non_vsf[[
        'bank_loan_interest',
        'sacco_loan_interest',
        'rtv_vsla_loan_interest',
        'other_village_VSLA_loan_interest'
        ]].mean()).transpose()

bank_loan_interest                  3.868421
sacco_loan_interest                 4.178788
rtv_vsla_loan_interest              3.591497
other_village_VSLA_loan_interest    5.860656
dtype: float64

In [522]:
(df_vsla[[
        'bank_loan_interest',
        'sacco_loan_interest',
        'rtv_vsla_loan_interest',
        'other_village_VSLA_loan_interest'
        ]].mean()).transpose()

bank_loan_interest                  3.857143
sacco_loan_interest                 3.967500
rtv_vsla_loan_interest              3.379409
other_village_VSLA_loan_interest    5.902985
dtype: float64

In [ ]:
df_vsla

## Loan Types

In [145]:
(rtv_vsf.groupby('state')['bank_loan_type'].value_counts(normalize=True) * 100).unstack().transpose()

state,non vsf,vsf
bank_loan_type,,
1.0,47.368421,100.0
2.0,52.631579,NaN


### Aggregating by Gender

In [575]:
non_vsf['Gender'].value_counts()

Gender
Male      1006
Female     353
Name: count, dtype: int64

In [576]:
adjusted_cpi = 80.66
rate = 3600

In [577]:
# Total Savings
((non_vsf.groupby(['Household_Head_Gender'])['Total Savings (Ugx)'].mean() * (adjusted_cpi/100))/rate)

Household_Head_Gender
Female Headed    60.118132
Male Headed      79.820095
Youth Headed     75.434234
Name: Total Savings (Ugx), dtype: float64

In [578]:
# Total Savings
((vsf.groupby(['Household_Head_Gender'])['Total Savings (Ugx)'].mean() * (adjusted_cpi/100))/rate)

Household_Head_Gender
Female Headed    79.637747
Male Headed      97.454326
Youth Headed     81.332167
Name: Total Savings (Ugx), dtype: float64

In [579]:
## Loan Amount for only those that saved
((subset.groupby(['state','Household_Head_Gender'])['Loan Amount (Ugx)'].mean() * (adjusted_cpi/100))/rate).unstack()

Household_Head_Gender,Female Headed,Male Headed,Youth Headed
state,,,
non vsf,54.069256,109.103818,38.985667
vsf,63.108981,99.049465,44.811111


In [580]:
for frame in [vsf, non_vsf]:
    for gender in frame['Household_Head_Gender'].unique():
        sub_frame = frame[frame['Household_Head_Gender'] == gender]
        debt = sub_frame[columns].sum(axis=1).mean() * (adjusted_cpi / 100) / rate
        print(f"{get_df_name(frame)} ({gender}):\t{debt}")

vsf (Female Headed):	14.411323350694444
vsf (Male Headed):	26.349993055555558
vsf (Youth Headed):	6.721666666666667
non_vsf (Male Headed):	29.57245874318769
non_vsf (Female Headed):	12.754094282769728
non_vsf (Youth Headed):	12.336235294117646


In [581]:
(vsf.groupby(['Household_Head_Gender'])['borrowed_past_12_months'].value_counts(normalize=True)).unstack().transpose()

Household_Head_Gender,Female Headed,Male Headed,Youth Headed
borrowed_past_12_months,,,
1,1.0,1.0,1.0


In [582]:
(non_vsf.groupby(['Household_Head_Gender'])['borrowed_past_12_months'].value_counts(normalize=True)).unstack().transpose()

Household_Head_Gender,Female Headed,Male Headed,Youth Headed
borrowed_past_12_months,,,
0,0.692754,0.662791,0.720588
1,0.307246,0.337209,0.279412


In [583]:
for frame in [vsf, non_vsf]:
    for gender in frame['Household_Head_Gender'].unique():
        sub_frame = frame[(frame['Household_Head_Gender'] == gender) & (frame['Interest'] != 0.)]
        interest = sub_frame['Interest'].mean()
        print(f"{get_df_name(frame)} (Gender: {gender}):\t{interest}")

vsf (Gender: Female Headed):	2.1422222222222222
vsf (Gender: Male Headed):	2.285849056603774
vsf (Gender: Youth Headed):	2.5
non_vsf (Gender: Male Headed):	4.151792452830189
non_vsf (Gender: Female Headed):	4.334615384615384
non_vsf (Gender: Youth Headed):	4.24


In [159]:
((vsf.groupby(['Household_Head_Gender'])['purpose_for_borrow'].value_counts(normalize=True)).unstack()).transpose()

Household_Head_Gender,Female Headed,Male Headed,Youth Headed
purpose_for_borrow,,,
1.0,0.222222,0.245283,NaN
3.0,0.055556,0.207547,0.5
4.0,0.444444,0.264151,NaN
6.0,0.277778,0.188679,0.5
7.0,NaN,0.037736,NaN
8.0,NaN,0.056604,NaN


In [160]:
((non_vsf.groupby(['Household_Head_Gender'])['purpose_for_borrow'].value_counts(normalize=True)).unstack()).transpose()

Household_Head_Gender,Female Headed,Male Headed,Youth Headed
purpose_for_borrow,,,
1.0,0.226415,0.326019,0.210526
2.0,0.037736,0.043887,0.052632
3.0,0.056604,0.084639,0.157895
4.0,0.235849,0.197492,0.105263
5.0,0.047170,0.031348,NaN
6.0,0.264151,0.206897,0.368421
7.0,0.018868,0.034483,0.052632
8.0,0.084906,0.040752,NaN
9.0,0.018868,0.018809,0.052632


In [155]:
(vsf.groupby('Household_Head_Gender')[[
    'Personal Business & Self Employment (USD_Cons_rate)',
    'Seasonal Crops Income (USD_Cons_rate)',
    'Seasonal  Agriculture Value (USD_Cons_rate)',
    'Livestock Income (USD_Cons_rate)',
    'Assets (USD_Cons_rate)',
    'HH Income (USD_Cons_rate)',
    'HH Income + Consumption + Residues (USD_Cons_rate)',
    'HH Income/Day (USD_Cons_rate)',
]].mean()).transpose()

Household_Head_Gender,Female Headed,Male Headed,Youth Headed
Personal Business & Self Employment (USD_Cons_rate),172.921877,169.951437,237.722944
Seasonal Crops Income (USD_Cons_rate),148.323604,142.028327,95.411847
Seasonal Agriculture Value (USD_Cons_rate),439.128059,439.876358,331.362483
Livestock Income (USD_Cons_rate),40.193466,35.270583,10.418583
Assets (USD_Cons_rate),233.623428,240.827174,278.949167
HH Income (USD_Cons_rate),637.812648,642.810393,593.937669
HH Income + Consumption + Residues (USD_Cons_rate),862.502028,906.689326,629.606356
HH Income/Day (USD_Cons_rate),1.747432,1.761124,1.627226


In [156]:
(non_vsf.groupby('Household_Head_Gender')[[
    'Personal Business & Self Employment (USD_Cons_rate)',
    'Seasonal Crops Income (USD_Cons_rate)',
    'Seasonal  Agriculture Value (USD_Cons_rate)',
    'Livestock Income (USD_Cons_rate)',
    'Assets (USD_Cons_rate)',
    'HH Income (USD_Cons_rate)',
    'HH Income + Consumption + Residues (USD_Cons_rate)',
    'HH Income/Day (USD_Cons_rate)',
]].mean()).transpose()

Household_Head_Gender,Female Headed,Male Headed,Youth Headed
Personal Business & Self Employment (USD_Cons_rate),69.633934,91.750179,121.086858
Seasonal Crops Income (USD_Cons_rate),142.828585,121.324301,113.668490
Seasonal Agriculture Value (USD_Cons_rate),363.211986,318.933753,297.786143
Livestock Income (USD_Cons_rate),24.533759,30.200865,18.758063
Assets (USD_Cons_rate),227.444964,242.915420,182.753550
HH Income (USD_Cons_rate),448.687551,509.229887,494.988314
HH Income + Consumption + Residues (USD_Cons_rate),793.744309,848.893293,806.666396
HH Income/Day (USD_Cons_rate),1.229281,1.395150,1.356132


In [199]:
a = np.array([2,3,4,5])
b = np.array([100,350,300,150])

In [203]:
c = b/a
c.mean()

67.91666666666667

In [201]:
b.mean()/a.mean()

64.28571428571429